In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q torch sentence_transformers datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.5 MB/s eta 0:00:00


In [3]:
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers import InputExample
import logging
from datetime import datetime
import os
import json
import math

In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import json

# train_samples = []
dict_data_list = []
exceptions_list = []

df = pd.read_csv('/content/drive/MyDrive/ibn_kathir_similarity.csv')

for file in os.listdir('/content/drive/MyDrive/corpus'):

  file_path = '/content/drive/MyDrive/corpus/' + file

  with open(file_path, 'r') as f:
    content = f.read()

  soup= BeautifulSoup(content, 'xml')

  for index, row in df.iterrows():

    try:
      source_address = 's' + row['Source_Sura'] + '.' + row['Source_Verse']
      target_address = 's' + row['target_sura'] + '.' + row['Target_verse']

      sentence1 = soup.find(id=source_address).contents[0]
      sentence1 = re.sub("[\(\[].*?[\)\]]", "", sentence1)
      sentence1 = re.sub(" +", " ", sentence1)

      sentence2 = soup.find(id=target_address).contents[0]
      sentence2 = re.sub("[\(\[].*?[\)\]]", "", sentence2)
      sentence2 = re.sub(" +", " ", sentence2)

      score = float(row['relevance_degree']) / 2.0

      # train_samples.append(InputExample(texts=[sentence1, sentence2], label=score))

      data_dict = {}
      data_dict['sentence1'] = sentence1
      data_dict['sentence2'] = sentence2
      data_dict['score'] = score
      dict_data_list.append(data_dict)

    except:
      pass


with open('/content/drive/MyDrive/verses_similarity.json', 'w') as f:
  json.dump(dict_data_list, f, ensure_ascii = False)

In [6]:
len(dict_data_list)

84421

In [4]:
import math
import random

with open('/content/drive/MyDrive/verses_similarity.json', 'r', encoding = 'utf-8-sig') as f:
  dict_data_list = json.load(f)

random.shuffle(dict_data_list)

split_per = math.trunc(0.8 * len(dict_data_list))

train_samples = []
dev_samples = []

for data in dict_data_list[:split_per]:
  train_samples.append(InputExample(texts=[data['sentence1'], data['sentence2']], label=data['score']))

for data in dict_data_list[split_per:]:
  dev_samples.append(InputExample(texts=[data['sentence1'], data['sentence2']], label=data['score']))

test_samples = dev_samples

# connect to hub

In [5]:
!pip install -q huggingface_hub

In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
# Define our Cross-Encoder
train_batch_size = 16
num_epochs = 1
model_save_path = "/content/drive/MyDrive/cross_encoder/cross_encoder-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

model = CrossEncoder("NeginShams/cross_encoder_v1", num_labels=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/853 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [9]:
# We wrap train_samples (which is a List[InputExample]) into a pytorch DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

# We add an evaluator, which evaluates the performance during training
evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name="sts-dev")

In [10]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up

In [11]:
# Train the model
model.fit(
    train_dataloader=train_dataloader,
    evaluator=evaluator,
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4221 [00:00<?, ?it/s]

In [12]:
##### Load model and eval on test set
model = CrossEncoder(model_save_path)

evaluator = CECorrelationEvaluator.from_input_examples(test_samples, name="sts-test")
evaluator(model)

0.7442195472199232

In [13]:
model.push_to_hub("cross_encoder_v2")

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NeginShams/cross_encoder_v2/commit/395ba3e6ee56dd8674bc836ab021861233e2da5a', commit_message='Upload CrossEncoder', commit_description='', oid='395ba3e6ee56dd8674bc836ab021861233e2da5a', pr_url=None, pr_revision=None, pr_num=None)